# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f789c3e1d90>
├── 'a' --> tensor([[-0.0613, -2.1107,  0.8647],
│                   [ 0.5007, -0.7210,  1.8474]])
└── 'x' --> <FastTreeValue 0x7f789c3e1d00>
    └── 'c' --> tensor([[ 2.5526, -1.4748, -0.9513,  0.4967],
                        [-0.1044, -1.1578,  0.9997,  1.5735],
                        [-0.5335,  0.2146,  0.9833,  1.0815]])

In [4]:
t.a

tensor([[-0.0613, -2.1107,  0.8647],
        [ 0.5007, -0.7210,  1.8474]])

In [5]:
%timeit t.a

62.4 ns ± 0.0495 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f789c3e1d90>
├── 'a' --> tensor([[ 2.5965, -0.2563, -0.0853],
│                   [-1.7906,  0.2729, -0.1823]])
└── 'x' --> <FastTreeValue 0x7f789c3e1d00>
    └── 'c' --> tensor([[ 2.5526, -1.4748, -0.9513,  0.4967],
                        [-0.1044, -1.1578,  0.9997,  1.5735],
                        [-0.5335,  0.2146,  0.9833,  1.0815]])

In [7]:
%timeit t.a = new_value

57.2 ns ± 0.0212 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0613, -2.1107,  0.8647],
               [ 0.5007, -0.7210,  1.8474]]),
    x: Batch(
           c: tensor([[ 2.5526, -1.4748, -0.9513,  0.4967],
                      [-0.1044, -1.1578,  0.9997,  1.5735],
                      [-0.5335,  0.2146,  0.9833,  1.0815]]),
       ),
)

In [10]:
b.a

tensor([[-0.0613, -2.1107,  0.8647],
        [ 0.5007, -0.7210,  1.8474]])

In [11]:
%timeit b.a

58.6 ns ± 0.0314 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1053, -1.0435,  0.1064],
               [-0.3945,  0.0985, -0.1531]]),
    x: Batch(
           c: tensor([[ 2.5526, -1.4748, -0.9513,  0.4967],
                      [-0.1044, -1.1578,  0.9997,  1.5735],
                      [-0.5335,  0.2146,  0.9833,  1.0815]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.154 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

835 ns ± 2.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 29.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 375 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 839 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f77e5d72430>
├── 'a' --> tensor([[[-0.0613, -2.1107,  0.8647],
│                    [ 0.5007, -0.7210,  1.8474]],
│           
│                   [[-0.0613, -2.1107,  0.8647],
│                    [ 0.5007, -0.7210,  1.8474]],
│           
│                   [[-0.0613, -2.1107,  0.8647],
│                    [ 0.5007, -0.7210,  1.8474]],
│           
│                   [[-0.0613, -2.1107,  0.8647],
│                    [ 0.5007, -0.7210,  1.8474]],
│           
│                   [[-0.0613, -2.1107,  0.8647],
│                    [ 0.5007, -0.7210,  1.8474]],
│           
│                   [[-0.0613, -2.1107,  0.8647],
│                    [ 0.5007, -0.7210,  1.8474]],
│           
│                   [[-0.0613, -2.1107,  0.8647],
│                    [ 0.5007, -0.7210,  1.8474]],
│           
│                   [[-0.0613, -2.1107,  0.8647],
│                    [ 0.5007, -0.7210,  1.8474]]])
└── 'x' --> <FastTreeValue 0x7f77e5680f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 66.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f77e56ceb20>
├── 'a' --> tensor([[-0.0613, -2.1107,  0.8647],
│                   [ 0.5007, -0.7210,  1.8474],
│                   [-0.0613, -2.1107,  0.8647],
│                   [ 0.5007, -0.7210,  1.8474],
│                   [-0.0613, -2.1107,  0.8647],
│                   [ 0.5007, -0.7210,  1.8474],
│                   [-0.0613, -2.1107,  0.8647],
│                   [ 0.5007, -0.7210,  1.8474],
│                   [-0.0613, -2.1107,  0.8647],
│                   [ 0.5007, -0.7210,  1.8474],
│                   [-0.0613, -2.1107,  0.8647],
│                   [ 0.5007, -0.7210,  1.8474],
│                   [-0.0613, -2.1107,  0.8647],
│                   [ 0.5007, -0.7210,  1.8474],
│                   [-0.0613, -2.1107,  0.8647],
│                   [ 0.5007, -0.7210,  1.8474]])
└── 'x' --> <FastTreeValue 0x7f77e56ceeb0>
    └── 'c' --> tensor([[ 2.5526, -1.4748, -0.9513,  0.4967],
                        [-0.1044, -1.1578,  0.9997,  1.5735],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 92.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.2 µs ± 41.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.0613, -2.1107,  0.8647],
                [ 0.5007, -0.7210,  1.8474]],
       
               [[-0.0613, -2.1107,  0.8647],
                [ 0.5007, -0.7210,  1.8474]],
       
               [[-0.0613, -2.1107,  0.8647],
                [ 0.5007, -0.7210,  1.8474]],
       
               [[-0.0613, -2.1107,  0.8647],
                [ 0.5007, -0.7210,  1.8474]],
       
               [[-0.0613, -2.1107,  0.8647],
                [ 0.5007, -0.7210,  1.8474]],
       
               [[-0.0613, -2.1107,  0.8647],
                [ 0.5007, -0.7210,  1.8474]],
       
               [[-0.0613, -2.1107,  0.8647],
                [ 0.5007, -0.7210,  1.8474]],
       
               [[-0.0613, -2.1107,  0.8647],
                [ 0.5007, -0.7210,  1.8474]]]),
    x: Batch(
           c: tensor([[[ 2.5526, -1.4748, -0.9513,  0.4967],
                       [-0.1044, -1.1578,  0.9997,  1.5735],
                       [-0.5335,  0.2146,  0.9833,  1.0815]],
         

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.0613, -2.1107,  0.8647],
               [ 0.5007, -0.7210,  1.8474],
               [-0.0613, -2.1107,  0.8647],
               [ 0.5007, -0.7210,  1.8474],
               [-0.0613, -2.1107,  0.8647],
               [ 0.5007, -0.7210,  1.8474],
               [-0.0613, -2.1107,  0.8647],
               [ 0.5007, -0.7210,  1.8474],
               [-0.0613, -2.1107,  0.8647],
               [ 0.5007, -0.7210,  1.8474],
               [-0.0613, -2.1107,  0.8647],
               [ 0.5007, -0.7210,  1.8474],
               [-0.0613, -2.1107,  0.8647],
               [ 0.5007, -0.7210,  1.8474],
               [-0.0613, -2.1107,  0.8647],
               [ 0.5007, -0.7210,  1.8474]]),
    x: Batch(
           c: tensor([[ 2.5526, -1.4748, -0.9513,  0.4967],
                      [-0.1044, -1.1578,  0.9997,  1.5735],
                      [-0.5335,  0.2146,  0.9833,  1.0815],
                      [ 2.5526, -1.4748, -0.9513,  0.4967],
                      [-0.1044, -

In [28]:
%timeit Batch.cat(batches)

140 µs ± 347 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

320 µs ± 1.54 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
